In [45]:
from helper import get_input

INPUTS = get_input(day=4)

TEST_INPUTS = """
..@@.@@@@.
@@@.@.@.@@
@@@@@.@.@@
@.@@@@..@.
@@.@@@@.@@
.@@@@@@@.@
.@.@.@.@@@
@.@@@.@@@@
.@@@@@@@@.
@.@.@@@.@.
"""

In [46]:
def prep_grid(inputs: str) -> list[list[str]]:
    return [list(x) for x in inputs.strip().split("\n")]


INPUTS_PREPPED = prep_grid(INPUTS)
TEST_INPUTS_PREPPED = prep_grid(TEST_INPUTS)

In [47]:
TEST_INPUTS_PREPPED

[['.', '.', '@', '@', '.', '@', '@', '@', '@', '.'],
 ['@', '@', '@', '.', '@', '.', '@', '.', '@', '@'],
 ['@', '@', '@', '@', '@', '.', '@', '.', '@', '@'],
 ['@', '.', '@', '@', '@', '@', '.', '.', '@', '.'],
 ['@', '@', '.', '@', '@', '@', '@', '.', '@', '@'],
 ['.', '@', '@', '@', '@', '@', '@', '@', '.', '@'],
 ['.', '@', '.', '@', '.', '@', '.', '@', '@', '@'],
 ['@', '.', '@', '@', '@', '.', '@', '@', '@', '@'],
 ['.', '@', '@', '@', '@', '@', '@', '@', '@', '.'],
 ['@', '.', '@', '.', '@', '@', '@', '.', '@', '.']]

## Part 1

In [48]:
def count_for_roll(inputs: list[list[str]], x: int, y: int) -> int:
    # At most 3 strings in lines
    lines = inputs[max(0, x - 1) : min(len(inputs), x + 2)]
    segments = [x[max(0, y - 1) : min(len(lines[0]), y + 2)] for x in lines]
    return [x for y in segments for x in y].count("@")


def part1(inputs: list[list[str]]) -> int:
    result = 0

    for x, line in enumerate(inputs):
        for y, char in enumerate(line):
            if char != "@":
                continue
            result += 1 if count_for_roll(inputs=inputs, x=x, y=y) < 5 else 0

    return result


print("EXPECTED:", 13)
print("TEST:    ", part1(TEST_INPUTS_PREPPED))
print("REAL:    ", part1(INPUTS_PREPPED))

EXPECTED: 13
TEST:     13
REAL:     1393


## Part 2

This seems fairly straightforward. I'll run through the grid and remove things (change them to `x` for instance),
setting a flag on one pass through the grid to indicate something was removed.
On a subsequent full pass, if no removals occur, then we know we have the result.

At first I was thinking we had to keep track of the removals on a whole pass, then go around again and actually remove them.
On the contrary, the process should be idempotent: items removed free up other items to be removed,
so we can freely remove anything fitting our criteria when we find it.
That may, in turn, let us remove more items per-pass through the grid,
meaning fewer passes overall.

In [49]:
import copy


def remove_roll(inputs: list[list[str]], x: int, y: int) -> None:
    inputs[x][y] = "x"


def part2(inputs: list[list[str]]) -> int:
    # TODO: below is just part 1 copied
    # Write the real thing according to my spec above.
    result = 0

    copied_inputs = copy.deepcopy(inputs)

    num_passes = 0
    while True:
        num_passes += 1
        map_changed = False
        for x, line in enumerate(copied_inputs):
            for y, char in enumerate(line):
                if char != "@":
                    continue
                if count_for_roll(inputs=copied_inputs, x=x, y=y) < 5:
                    result += 1
                    remove_roll(inputs=copied_inputs, x=x, y=y)
                    map_changed = True
        if not map_changed:
            print(f">> Number of passes through the grid: {num_passes}")
            break
    return result


print("EXPECTED:", 43)
print("TEST:    ", part2(TEST_INPUTS_PREPPED))
print("REAL:    ", part2(INPUTS_PREPPED))

EXPECTED: 43
>> Number of passes through the grid: 4
TEST:     43
>> Number of passes through the grid: 41
REAL:     8643


In hindsight, in part 1 I originally just had a list of strings, since I was counting instances and didn't have to do much replacement.

On attempting part 2, I realized manipulating those large strings would be costly, so I opted instead to prep the grids be splitting them into lists of lists of characters. That way, I could just assign individual characters to the grid at the given location.

Finally, the same `count_for_roll` method was used (slightly modified to work on the new data type and flatten nested lists in order to count occurrences), but I just had to add the `remove_roll` method on top of it.

I probably could have done something more efficient if I cared to, but again, 0.1s working time. Brute force works just fine here.